# Langchain doc retriever with ChromaDB

get the OpenAI key from .env file

In [2]:
from dotenv import load_dotenv
# Import the function load_dotenv from the dotenv package.

import os
# Import the os module for interacting with the operating system.

load_dotenv()
# Load environment variables from a .env file.

openai_api_key = os.getenv('OPENAI_API_KEY')
# Get the value of the 'OPENAI_API_KEY' environment variable.

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.document_loaders import TextLoader

In [12]:
# Load and process the text files
loader = TextLoader('papers.txt')

papers = loader.load()

In [17]:
# splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(papers) 
# every chunk has 1000 chracters
# below we see the features of OpenAI model = text-embedding-ada-002 and chunk_size=1000
# chunk_overlap iki chuns arasi ortak karakter sayisi
len(texts) # We have 114 chunks

114

# Create ChromaDB

In [18]:
# Embed and store the text
# Supplying a persist directory will store the embeddings on disk
persist_directory = 'db'

In [22]:
embedding = OpenAIEmbeddings()   
# OpenAI embeddings modelini kullaniyoruz. onun kendine ait embeddings kümesi var ve tokenizer
# bununla PDF ler ele aliniyor ve anlamlari ele aliniyor

In [24]:
vectordb = Chroma.from_documents(documents = texts, # bizim PDF lerin toplami
                                 embedding = embedding, 
                                 persist_directory = persist_directory)
                                 # owerwrite parametreside var. True ise m,evcut verilerin üstüne yazar, False ise yeni bir database olusturur

# burda texts embeddings ile vectörlere cevrilir ve Chroma db e kaydedilir
# ve calistirinca bir 'db' adinda bir database olusturulur ve icine kaydedilir
# tüm belgelerimizi kodladi yani Openai embeddinginden gecti.database'e kaydedilme burda yapiliyor

In [25]:
# persiste the db to disk
vectordb.persist()  # bu vector database diske kaydeder. yani datalari bir sürü diske kaydeder. database in boyutunu belirler ve icindekileri diske yazar
vectordb = None # yukarida diske kaydedildigi icin bu degiskenin artik kullanilmayacagini gösterir.

# vector database size'ini belirlemek icin vector database 'collection' ve documents özelliklerini kullanir
# documents bizim kullandigimiz degisken icinde tüm textin oldugu

# bunlari kaydedersekiyi olur. cünkü kaydetmezsek bu uygulamayi her actigimizda tekrardan Openai embeddingi yapmak gerekecek
# kaydedersek ordan direkt kullanilir. 
# aslinda bu kaydetme islemini bir kere yazariz ve owerwrite parametresini ona göre ayarlayabiliriz

In [27]:
# now we can load the persisted database from disk and use it as normal
vectordb = Chroma(persist_directory = persist_directory,
                  embedding_function = embedding)

# daha önce diske kaydettigimiz Chromadb yi tekrardan kullanmak icin yapiyoruz

#### make a retriever

In [28]:
# burda aslinda byte yapmak istedigi sey . ilgili sorunun dosyasini buluyor.
# hem cevabin oldugu metni veriyor ve en altta dosyanin adinida veriyor
# burda metinler chunks lar halinde geliyor. her 1000 charakter 1 chunks oluyor

In [29]:
retriever = vectordb.as_retriever()   # bu database'i bir arama motoruna cevirir. ve metin aramsi yapmak icin kullanilir
# bir üsttekiyle retriever baglantili hale getiriliyor. baglantili haliyle bir asagida yeni bir function icinde kullanilabiliyor
docs = retriever.get_relevant_documents('For which tasks has Llama-2 already been used successfully? What are promising areas of application for Llama-2?') # bu bir liste döndürür. burda bir degisken adi yazip, yularida bu degisken adinin oldugu yere soruyu yazabiliriz, daha okunakli olur
docs

# bu retriever asagida chain olusturulken kullanacagiz

[Document(page_content='Title: Llama 2: Open Foundation and Fine-Tuned Chat Models\nSummary:   In this work, we develop and release Llama 2, a collection of pretrained and\nfine-tuned large language models (LLMs) ranging in scale from 7 billion to 70\nbillion parameters. Our fine-tuned LLMs, called Llama 2-Chat, are optimized for\ndialogue use cases. Our models outperform open-source chat models on most\nbenchmarks we tested, and based on our human evaluations for helpfulness and\nsafety, may be a suitable substitute for closed-source models. We provide a\ndetailed description of our approach to fine-tuning and safety improvements of\nLlama 2-Chat in order to enable the community to build on our work and\ncontribute to the responsible development of LLMs.', metadata={'source': 'papers.txt'}),
 Document(page_content='Title: Financial News Analytics Using Fine-Tuned Llama 2 GPT Model\nSummary:   The paper considers the possibility to fine-tune Llama 2 GPT large language\nmodel (LLM) for 

In [30]:
len(docs)
# burdanda 4 tane chunks oldugunu görebiliyorum

4

# Make a Chain

In [31]:
# create the chain to answer questions
# burda bir tanimlama yaoacagiz ve asagida sorgu yaparken kullanacagiz

qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                        chain_type='stuff',
                                        retriever=retriever,
                                        return_source_documents = True)

# qa_chain = bunu yazdirirsak icinde Retrival ile bilgileri yazdiriyor
# openAi killandigimizi gösteriyor ve open_key gibi biligler veriyor

In [32]:
query = 'For which tasks has Llama-2 already been used successfully? What are promising areas of application for Llama-2?'

qa_chain(query)
# burda gelen cevabin icinde aslinda nelerin oldugunu görebiliyoruz
# query
# result
# source_documents

{'query': 'For which tasks has Llama-2 already been used successfully? What are promising areas of application for Llama-2?',
 'result': ' Llama 2 has been successfully used for dialogue and financial news analysis tasks. Promising areas of application for Llama 2 include content moderation, language classification, and AI safety.',
 'source_documents': [Document(page_content='Title: Llama 2: Open Foundation and Fine-Tuned Chat Models\nSummary:   In this work, we develop and release Llama 2, a collection of pretrained and\nfine-tuned large language models (LLMs) ranging in scale from 7 billion to 70\nbillion parameters. Our fine-tuned LLMs, called Llama 2-Chat, are optimized for\ndialogue use cases. Our models outperform open-source chat models on most\nbenchmarks we tested, and based on our human evaluations for helpfulness and\nsafety, may be a suitable substitute for closed-source models. We provide a\ndetailed description of our approach to fine-tuning and safety improvements of\nL

In [42]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')

    if llm_response['source_documents']:
        source = llm_response['source_documents'][0]
        print(source.metadata['source'])


# Example Questions

Question 1:

In [43]:
query = 'For which tasks has Llama-2 already been used successfully? What are promising areas of application for Llama-2?'
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Llama-2 has been successfully used for fine-tuning LLMs for dialogue use cases, financial news analytics, and safety training. Promising areas of application for Llama-2 include financial news analysis, content moderation, and AI safety.


Sources:
papers.txt


Question 2:

In [44]:
query = 'Name at least 5 domain-specific LLMs that have been created by fine-tuning Llama-2.'
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Five domain-specific LLMs that have been created by fine-tuning Llama-2 are Lawyer LLaMA, Llama 2-Chat, Llama 2 GPT, Llama 2 model for financial news analysis, and LLaMA for specific writing tasks.


Sources:
papers.txt


Question 3:

In [45]:
query = 'What can you find out about the model structure of Llama-2 (required memory, required computing capacity, number of parameters, available quantizations)?'
llm_response = qa_chain(query)
process_llm_response(llm_response)



Based on the provided context, the Llama-2 model has a range of scales from 7 billion to 70 billion parameters. The fine-tuned Llama 2-Chat models are optimized for dialogue use cases and outperform open-source chat models on most benchmarks. The Code Llama family of models, based on Llama 2, also have different variations such as foundation models, Python specializations, and instruction-following models with 7B, 13B, and 34B parameters each. These models are trained on sequences of 16k tokens and show improvements on inputs with up to 100k tokens. The 7B and 13B Code Llama and Code Llama - Instruct variants also support infilling based on surrounding content. The models achieved state-of-the-art performance among open models on several code benchmarks, with scores of up to 53% and 55% on HumanEval and MBPP, respectively. The required memory and computing capacity for these models may vary depending on the specific variant and task at hand. The available quantizations for these mode